# Reproducing NY Fed Staff Report 1111: Financing Private Credit

**Paper**: Boyarchenko, N., & Elias, L. (2024). *Financing Private Credit*. Federal Reserve Bank of New York Staff Reports, No. 1111.

**Key Findings**:
1. The sectoral composition of lenders financing a credit expansion determines subsequent real activity and crisis probability
2. Banks and nonbanks respond differentially to macroeconomic conditions
3. Bank credit is more sensitive to economic downturns than nonbank credit

This notebook reproduces the methodology and extends it with nowcasting capabilities.

In [ ]:
import sys
sys.path.insert(0, '../src')

import polars as pl
import altair as alt
import numpy as np

from financing_private_credit.data import PrivateCreditData, FREDDataFetcher
from financing_private_credit.analysis import CreditDecomposition, DemandSystemModel
from financing_private_credit.nowcast import CreditNowcaster, FinancialConditionsMonitor
from financing_private_credit import viz

# Configure display
alt.data_transformers.disable_max_rows()
pl.Config.set_tbl_rows(20)

## 1. Data Collection

Fetch private credit data from FRED (Z.1 Financial Accounts).

In [ ]:
# Initialize data fetcher with 1990 start date for modern credit cycle analysis
credit_data = PrivateCreditData(start_date="1990-01-01")

# Fetch all relevant series from FRED
raw_data = credit_data.fetch_all()
print(f"Data shape: {raw_data.shape}")
print(f"Columns: {raw_data.columns}")
raw_data.tail(10)

## 2. Credit Decomposition by Lender Type

Decompose total private credit into:
- **Bank credit**: Commercial banks, credit unions, savings institutions
- **Nonbank credit**: Shadow banks (MMFs, ABS issuers, broker-dealers, finance companies), insurance, pensions

In [ ]:
# Compute credit decomposition
decomposed = credit_data.compute_credit_decomposition()
print("Credit decomposition:")
decomposed.tail(10)

In [ ]:
# Compute lender shares
shares = credit_data.get_lender_shares()
print("Lender shares:")
shares.select(["date", "bank_share", "nonbank_share"]).tail(10)

In [ ]:
# Visualize lender composition over time
if "bank_share" in shares.columns and "nonbank_share" in shares.columns:
    viz.chart_lender_composition(shares)

## 3. Credit-to-GDP Analysis

Normalize credit by GDP to analyze credit cycles following Schularick & Taylor (2012).

In [ ]:
# Compute credit-to-GDP ratios
try:
    credit_gdp = credit_data.compute_credit_to_gdp()
    print("Credit-to-GDP ratios:")
    ratio_cols = [c for c in credit_gdp.columns if c.endswith("_to_gdp")]
    credit_gdp.select(["date"] + ratio_cols).tail(10)
except ValueError as e:
    print(f"Note: {e}")
    credit_gdp = None

In [ ]:
# Visualize credit-to-GDP trends
if credit_gdp is not None:
    viz.chart_credit_to_gdp(credit_gdp)

## 4. Demand System Approach

Estimate credit supply elasticities to understand how different lender types respond to economic conditions.

In [ ]:
# Prepare data for demand system analysis
analysis_data = decomposed.drop_nulls()

if analysis_data.height > 0 and "gdp" in analysis_data.columns:
    # Initialize demand system model
    model = DemandSystemModel(analysis_data)
    
    # Estimate elasticities
    results = model.estimate_full_system()
    
    print("\n=== Demand System Estimation Results ===")
    print(f"Bank credit elasticity to output: {results['supply_elasticities'].bank_elasticity:.3f}")
    print(f"  (Standard error: {results['supply_elasticities'].bank_se:.3f})")
    print(f"\nNonbank credit elasticity to output: {results['supply_elasticities'].nonbank_elasticity:.3f}")
    print(f"  (Standard error: {results['supply_elasticities'].nonbank_se:.3f})")
    print(f"\nBank credit more procyclical: {results['bank_more_procyclical']}")
    print(f"\nObservations: {results['supply_elasticities'].n_obs}")
else:
    print("Insufficient data for elasticity estimation")
    results = None

In [ ]:
# Visualize elasticity comparison
if results is not None:
    e = results['supply_elasticities']
    viz.chart_elasticity_comparison(
        bank_elasticity=e.bank_elasticity,
        nonbank_elasticity=e.nonbank_elasticity,
        bank_se=e.bank_se,
        nonbank_se=e.nonbank_se,
        title="Credit Supply Elasticities to Output"
    )

## 5. Cyclical Analysis

Analyze cyclical properties of bank vs nonbank credit.

In [ ]:
# Compute cyclical components
decomp_obj = CreditDecomposition(decomposed.drop_nulls())

try:
    cyclical = decomp_obj.compute_cyclical_properties()
    print("Cyclical components:")
    cycle_cols = [c for c in cyclical.columns if c.endswith("_cycle")]
    cyclical.select(["date"] + cycle_cols).tail(10)
except ValueError as e:
    print(f"Note: {e}")
    cyclical = None

In [ ]:
# Visualize cyclical behavior
if cyclical is not None and "bank_credit_cycle" in cyclical.columns:
    viz.chart_cyclical_comparison(cyclical)

## 6. Crisis Probability Indicator

Construct crisis risk indicator based on the paper's findings: credit expansions financed primarily by banks are associated with higher crisis probability.

In [ ]:
# Compute crisis probability indicator
if results is not None:
    crisis_data = model.compute_crisis_probability(
        credit_growth_threshold=10.0,  # 10% YoY growth
        bank_share_threshold=55.0,     # >55% bank financed
    )
    
    print("Crisis risk indicators:")
    risk_cols = ["date", "total_credit_growth", "bank_share", "elevated_crisis_risk"]
    available_cols = [c for c in risk_cols if c in crisis_data.columns]
    crisis_data.select(available_cols).tail(20)

## 7. Nowcasting Extension

Extend the quarterly analysis with higher-frequency (weekly) bank credit data for real-time monitoring.

In [ ]:
# Initialize nowcaster
nowcaster = CreditNowcaster(lookback_years=10)

# Fetch high-frequency proxy data (weekly H.8 bank credit)
print("Fetching weekly bank credit data...")
proxy_data = nowcaster.fetch_proxy_data()
print(f"\nProxy data shape: {proxy_data.shape}")
print(f"Date range: {proxy_data['date'].min()} to {proxy_data['date'].max()}")
proxy_data.tail(10)

In [ ]:
# Compute credit growth nowcast
growth_nowcast = nowcaster.compute_credit_growth_nowcast()
print("\nBank credit growth nowcast:")
growth_cols = [c for c in growth_nowcast.columns if "growth" in c.lower()]
growth_nowcast.select(["date"] + growth_cols).tail(10)

In [ ]:
# Monitor current financial conditions
monitor = FinancialConditionsMonitor()
conditions = monitor.assess_credit_environment()

print("\n=== Current Financial Conditions ===")
print(f"Assessment date: {conditions.get('date')}")
print(f"\nOverall credit environment: {conditions.get('overall', 'N/A').upper()}")

print("\nIndicator details:")
for name, details in conditions.get('indicators', {}).items():
    print(f"  {name}: {details.get('value', 'N/A'):.2f} ({details.get('interpretation', 'N/A')})")

## 8. Summary Dashboard

Create comprehensive visualization of private credit conditions.

In [ ]:
# Create summary dashboard if we have sufficient data
if decomposed.height > 0:
    dashboard = viz.create_dashboard(
        decomposed,
        title="Private Credit Monitor - US"
    )
    dashboard

## Key Takeaways

### Methodology Reproduction
1. **Data**: Used Z.1 Financial Accounts from FRED to decompose private credit by lender type
2. **Demand System**: Estimated supply elasticities showing bank credit is more procyclical
3. **Crisis Indicator**: Constructed risk metric based on credit growth + lender composition

### Nowcasting Extension
1. **Weekly Proxies**: H.8 bank credit data provides real-time signal
2. **Financial Conditions**: NFCI, credit spreads indicate current credit environment
3. **Bridge Equations**: Connect weekly proxies to quarterly Z.1 data

### Policy Implications (from paper)
- Secular shift from bank to nonbank financing affects credit cycle dynamics
- Bank-financed credit expansions carry higher crisis probability
- Monitoring lender composition is crucial for financial stability assessment